In [1]:
from ortools.sat.python import cp_model
import numpy as np

#test 1
d = np.array([[ 0,  1, 12,  9,  9,  3,  8,  6, 19, 10],
              [ 3,  0,  5,  8, 16, 16, 10,  3, 10, 14],
              [ 2, 11,  0, 12, 10,  7,  3,  7, 11, 16],
              [ 2,  4, 14,  0,  3, 16, 18, 10,  1, 19],
              [16, 15,  6,  8,  0, 17,  2, 20, 16, 16],
              [15, 10, 19,  6, 14,  0,  8,  9,  7, 12],
              [12,  2,  1,  9, 20, 15,  0,  1,  2,  7],
              [12,  2, 19,  6, 12, 17, 14,  0,  7,  2],
              [11, 15,  8,  4, 15, 15, 20, 19,  0, 10],
              [11, 19,  7, 18,  8, 20, 17,  9,  7,  0]])
            

'''
#test 2
d = np.array([[ 0, 46, 39, 20, 21, 39, 50, 45, 43, 44],
              [41,  0, 33, 45, 34, 36, 32, 30, 45, 36],
              [34, 30,  0, 23, 28, 39, 26, 26, 42, 27],
              [21, 35, 26,  0, 46, 21, 23, 35, 46, 20],
              [49, 42, 39, 40,  0, 48, 47, 36, 37, 41],
              [39, 21, 28, 42, 28,  0, 33, 33, 38, 42],
              [33, 28, 43, 48, 42, 40,  0, 23, 48, 28],
              [42, 26, 50, 40, 41, 50, 25,  0, 22, 49],
              [39, 35, 29, 31, 43, 43, 36, 46,  0, 36],
              [27, 31, 27, 41, 28, 40, 33, 50, 47,  0]])
              '''

N = 9
K = 3

#Hàm tính tổng của nested_list
def nested_sum(L):
    total = 0  
    for i in L:
        if isinstance(i, list):  
            total += nested_sum(i)
        else:
            total += i
    return total

def c(t):
    global N
    temp = t 
    if temp > N:
        return 0
    return temp

#Notations
B = list()
F1 = list()
F2 = list()
F3 = list()
F4 = list()
A = list()
A_plus = list()
A_minus = list()

for i in range(1, N+2*K+1):
    for j in range(1, N+2*K+1):
        B.append((i, j))

for i in range(1, N+2*K+1):
    for k in range(1, K+1):
        F1.append((i, N+k))
        
for i in range(1, N+2*K+1):
    for k in range(1, K+1):
        F2.append((N+K+k, i))
        
for i in range(1, N+2*K+1):
    F3.append((i, i))
    
for i in range(N+1, N+K+1):
    for j in range(N+K+1, N+2*K+1):
        F4.append((i, j))
    
for edge in B:
    if (edge not in F1) and (edge not in F2) and (edge not in F3) and (edge not in F4):
        A.append(edge)

A_plus.append([])
for i in range(1, N+2*K+1):
    sub_lst = list()
    for edge in A:
        if edge[0] == i:
            sub_lst.append(edge[1])
    A_plus.append(sub_lst)

A_minus.append([])
for i in range(1, N+2*K+1):
    sub_lst = list()
    for edge in A:
        if edge[1] == i:
            sub_lst.append(edge[0])
    A_minus.append(sub_lst)

model = cp_model.CpModel()

x = [[[model.NewIntVar(0,1,f'x[{k}][{i}][{j}]') for j in range(N + 2 * K + 1)]
                                                for i in range(N + 2 * K + 1)]
                                                for k in range(K + 1)]

#constraint 1
for k in range (1, K + 1):
    model.Add((nested_sum([x[k][k+N][j] for j in range(1, N + 1)])) == 1)
    model.Add((nested_sum([x[k][j][k+N+K] for j in range(1, N + 1)])) == 1)

#constraint 2
for k in range (1, K + 1):
        for i in range (N + 1, N + K + 1):
            if i != N + k:
                model.Add((nested_sum([x[k][i][j] for j in range (1, N + 1)])) == 0)
        for j in range (N + K + 1, N + 2 * K + 1):
            if j != N + K + k:
                model.Add((nested_sum([x[k][i][j] for i in range (1, N + 1)])) == 0)

#constraint 3
for i in range (1, N + 1):
    model.Add((nested_sum([[x[k][i][j] for j in A_plus[i]] for k in range (1, K + 1)])) == 1)
    model.Add((nested_sum([[x[k][j][i] for j in A_minus[i]] for k in range (1, K + 1)])) == 1)

#constraint 4
for i in range(1, N + 1):
    for k in range(1, K+1):
        model.Add((nested_sum([x[k][i][j] for j in A_plus[i]]) ==
                  (nested_sum([x[k][j][i] for j in A_minus[i]]))))

#constraint 5           
u = [[model.NewIntVar(0, N, f'u[{k}][{i}]') for i in range(N+2*K+1)] for k in range(K+1)]


for k in range(1, K+1):
    for i in range(N+1, N+2*K+1):
        model.Add(u[k][i] == 0)
        
for k in range(1, K+1):
    for i in range(1, N+1):
        for j in range(1, N+1):
            b = model.NewBoolVar('b')
            model.Add(x[k][i][j] == 1).OnlyEnforceIf(b)
            model.Add(x[k][i][j] != 1).OnlyEnforceIf(b.Not())
            model.Add(u[k][j] == u[k][i] + 1).OnlyEnforceIf(b)
            
for i in range(1, N+1):
    for j in range(1, N+1):
        b = model.NewBoolVar('b')
        model.Add(i != j).OnlyEnforceIf(b)
        model.Add(i == j).OnlyEnforceIf(b.Not())
        model.Add(u[k][i] - u[k][j] + N*x[k][i][j] <= N-1).OnlyEnforceIf(b)
        
#constraint to support Objective
z = model.NewIntVar(0, 1000000000000000000, 'z')

for k in range (1, K + 1):  
    model.Add(nested_sum([[x[k][i][j] * d[c(i)][c(j)] for j in range (1, N + K * 2 + 1)]
                                                for i in range (1, N + K * 2 + 1)]) <= z)
    
#Objective
model.Minimize(z * K + (nested_sum([[[x[k][i][j] * d[c(i)][c(j)] for j in range (1, N + K * 2 + 1)]
                                                for i in range (1, N + K * 2 + 1)]
                                                for k in range(1, K + 1)])))

solver = cp_model.CpSolver()
status = solver.Solve(model)
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    max_route = 0
    total_route = 0
    print(f'Minimum of objective function: {solver.ObjectiveValue()}\n')
    for k in range(1, K + 1):
        total = 0
        for i in range(N + 2 * K + 1):
            for j in range(N + 2 * K + 1): 
                    if solver.Value(x[k][i][j]) == 1:
                        print(f'x[{k}][{c(i)}][{c(j)}] = ', solver.Value(x[k][i][j]))
                        total += d[c(i)][c(j)]
        max_route = max(max_route, total)
        total_route += total
                       
    print('Total = ', total_route)
    print('Max_route = ', max_route)
elif status == cp_model.INFEASIBLE:
    print('No solution')

Minimum of objective function: 98.0

x[1][1][7] =  1
x[1][7][9] =  1
x[1][9][0] =  1
x[1][0][1] =  1
x[2][3][0] =  1
x[2][5][8] =  1
x[2][8][3] =  1
x[2][0][5] =  1
x[3][2][0] =  1
x[3][4][6] =  1
x[3][6][2] =  1
x[3][0][4] =  1
Total =  47
Max_route =  17
